# Capital Asset Pricing Model
Welcome to this easy-to-use CAPM program!
All you need to do is enter your desired exchange-traded ticker below and view the results printed by the last cell.

In [ ]:
symbol = 'ENTER TICKER HERE'

In [ ]:
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import requests

In [ ]:
r = requests.get('https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v2/accounting/od/avg_interest_rates?fields=record_date,security_desc,avg_interest_rate_amt&filter=security_desc:eq:Treasury Bills&sort=-record_date')

In [ ]:
raw = r.json()
data = raw['data']
rf_data = [round(float(i['avg_interest_rate_amt']) / 100, 4) for i in data]

rf_df = pd.DataFrame(rf_data[:61])
rf_df = rf_df.loc[::-1]

In [ ]:
stock = yf.Ticker(symbol)
market = yf.Ticker('spy')

mkt = market.history(period='5y', interval='1mo')
stk = stock.history(period='5y', interval='1mo')
mkt.dropna(inplace=True)
stk.dropna(inplace=True)

stk['rm'] = round(mkt['Close'].pct_change(), 4)
stk['rs'] = round(stk['Close'].pct_change(), 4)
stk['rf'] = rf_df.values
stk = stk.iloc[:60, -3:]
stk.dropna(inplace=True)

In [ ]:
capm_df = pd.DataFrame()
capm_df['market excess'] = stk['rm'] - stk['rf']
capm_df['stock excess'] = stk['rs'] - stk['rf']
capm_df.dropna(inplace=True)

In [ ]:
x = capm_df['market excess']
x_bar = x.mean()
y = capm_df['stock excess']
y_bar = y.mean()

x_0 = x - x_bar
y_0 = y - y_bar

n = x_0 * y_0
d = x_0 ** 2
q = n.sum() / d.sum()
r_s = (stk.iloc[-1, -1] + q * stk['rm'].mean()) * 12 ** 0.5

print(f"{symbol.upper()}'s beta: {round(q, 3)}")
print(f"Required rate of return: {round(r_s, 4) * 100}%")